In [17]:
import numpy as np
import torch
from torch.utils import data

true_w = torch.tensor([2, -3.4])
true_b = 4.2

def synthetic_data(w, b, num_example):
    """生成 y = Xw + b + 噪声。"""
    X = torch.normal(0, 1, (num_example, len(w)))  # 生成均值为0，标准差为1的随机数，行数为样本数，列数为w的长度
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)  # 随机噪声
    return X, y.reshape((-1, 1))  # 将y返回成列向量

features, labels = synthetic_data(true_w, true_b, 1000)

In [18]:
# 调用框架中现有的API来读取数据
def load_array(data_arrays, batch_size, is_train=True):
    """
    构造一个pyTorch数据迭代器
    """
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.1111, -0.3725],
         [ 1.4192,  1.6246],
         [ 0.8314,  0.6754],
         [ 0.5407,  1.7952],
         [ 0.1734, -1.5008],
         [-0.8199,  0.8850],
         [-0.7359, -1.3778],
         [-0.1123, -1.6092],
         [ 1.2742,  0.0461],
         [-0.6999,  1.0001]]),
 tensor([[ 5.6844],
         [ 1.5184],
         [ 3.5707],
         [-0.8394],
         [ 9.6570],
         [-0.4547],
         [ 7.4195],
         [ 9.4421],
         [ 6.5962],
         [-0.6068]])]

In [19]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))  # 2输入维度 和 1输出维度

# 初始化模型参数

In [20]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [21]:
"""
计算均方误差使用的是MSELoss类，也称平方L2范数
"""
loss = nn.MSELoss()

In [22]:
"""
实例化SGD实例
"""
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [23]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()  # 这里pyTorch已经计算了sum()，无需自己再计算
        trainer.step()  # 调用step()函数，从而分别更新 权重 和 偏差
    l = loss(net(features), labels)
    print(f'epoch{epoch+1}, loss{l:f}')

epoch1, loss0.000324
epoch2, loss0.000105
epoch3, loss0.000105
